In [1]:
import boto3
import boto3.session
import pandas as pd

In [2]:
def list_files(s3_path):
    """
    Autor: ADL
    Email: ADL
    Descripcion; Función para ver los elementos que hay en una ruta determinada.
    Parámetros: texto con la ruta de interés
    Retorno: Listado de los elementos que se encuentra en la ruta determinada.
    """
    s3 = boto3.resource('s3')
    bucket = s3_path.split('//')[1].split('/')[0]
    my_bucket = s3.Bucket(bucket)
    prefix = '/'.join(s3_path.split('//')[1].split('/')[1:])
    n_sub = len(prefix.split('/'))
    list_obj = []
    for object_summary in my_bucket.objects.filter(Prefix=prefix):
        obj_x = object_summary.key.split('/')[n_sub - 1]
        if not obj_x in list_obj:
            list_obj.append(obj_x)
    return(list_obj)

In [3]:
# list_files('s3://data-bpop-dev-sandbox/estandarizado/productos/libranzas/')
# list_files('s3://data-bpop-dev-sandbox/estandarizado/productos/activo/')
list_files('s3://data-commons-dev-sandbox-de/activos_propios/fuentes_externas/supersociedades/')

['consolidado',
 'plenas-individuales',
 'plenas-separados',
 'pymes-individuales',
 'pymes-separados']

## Process Preparation

In [4]:
input_paths_activos = { 'libranzas': ['s3://data-bpop-dev-sandbox/estandarizado/productos/libranzas/productos_libranzas_dwh_M202106'],  ## Mensual 1 mes atras respecto al mes que esta corriendo - coger v2
         'clientes': ['s3://data-bpop-dev-sandbox/estandarizado/clientes/identificacion/clientes_identificacion_mdm_D20210630'],  ## Mensual 1 mes atras - coger v2 #### NO SE NECESITA QUITAR

         'activo': ['s3://data-bpop-dev-sandbox/estandarizado/productos/activo/productos_activo_sarc_M202105'],  ## Mensual 2 meses atras - coger v2

         'canpañas': 's3://data-bpop-dev-refined/gestion-campanas/preaprobado-tc/',
         'crm': ['s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20200228-20200601.csv',  ## Quemado
                 's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190801-20200201.csv',  ## Quemado
                 's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190104-20190702.csv'],  ## Quemado
         'vendors':['s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20200401-20200601.csv',  ## Quemado
                    's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20190101-20200301.csv'],  ## Quemado

         'activaciones': ['s3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-activadas/productos_activos-tarjeta-credito-activadas_H202003-202005.csv',  ## Quemad o
                          's3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-activadas/productos_activos-tarjeta-credito-activadas_H20180101-20200327.csv'],  ## Quemado
         'acti_diarias': ['s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva'],  ##TodosDiarios

         'buro': 's3://data-bpop-dev-sandbox/estandarizado/riesgos-creditos/buro-comportamiento-financiero',  ##TodosDiarios

         'pasivo_cc': 's3://data-bpop-dev-sandbox/estandarizado/productos/pasivo-sfb-cc/productos_pasivo-sfb-cc_aaatr_M',

         'pasivo_cdt': 's3://data-bpop-dev-sandbox/estandarizado/productos/pasivo-sfb-cdts/productos_pasivo-sfb-cdts_aaatr_M',
                       
         'pasivo_ca': 's3://data-bpop-dev-sandbox/estandarizado/productos/pasivo-sfb-ca/productos_pasivo-sfb-ca_aaatr_M'
}

In [5]:
def preparation(input_paths_activos):
    
    libranzas_df = process_libranzas(input_paths_activos)
    print('process_libranzas_successfully')
    ids_final2 = processing_filtering(input_paths_activos)
    print('processing_filtering_successfully')
    data_activos = process_activos(input_paths_activos)
    print('process_activos_successfully')
    tabla_final = process_camp_activos(input_paths_activos)
    print('process_camp_activos_successfully')
    buro_mdt = process_buro(input_paths_activos)
    print('process_buro_successfully')
    pasvivos_df = process_pasivos(input_paths_activos)
    print('process_pasivos_successfully')
    
    poblacion_total = consolidado_premdt(ids_final2)
    print('consolidado_premdt_successfully')
    
    return libranzas_df, ids_final2, data_activos, tabla_final, buro_mdt, pasvivos_df, poblacion_total


In [21]:
#libranzas_df, ids_final2, data_activos, tabla_final, buro_mdt, pasvivos_df, poblacion_total = preparation(input_paths_activos)

In [23]:
#libranzas_df.head()

,id_numero_cliente,id_tp_cd,num_lib_solicitadas,prom_monto_novado,prom_n_cuotas,periodo
0,27079010,1000003,1,16502114.0,120.0,202104
1,14972564,1000003,1,6788899.0,48.0,202104
2,11222544,1000003,1,80338044.0,99.0,202104
3,27076024,1000003,1,17376076.0,92.0,202104
4,19207951,1000003,1,0.0,120.0,202104


In [24]:
ids_final2.head()

,id_cliente,fecha_camp,id_numero_cliente,id_tp_cd
0,102652297589965301,202101,27079010,1000003
1,102652296060855001,202101,14972564,1000003
2,102652304429247101,202101,80354687,1000003
3,102652295542372001,202101,11222544,1000003
4,102652297598623801,202101,27076024,1000003


In [25]:
data_activos.head()

,nroident,tipo_ident_hom,fecha_activos,amortizacion_min_lb,cuota_max_lb_smlv
0,206,1000003,202104,0.840337,0.531937
1,1112,1000003,202104,0.417281,0.312141
2,1272,1000003,202104,0.960803,1.563886
3,1974,1000003,202104,0.339418,0.835206
5,2460,1000003,202104,0.512831,1.785160


In [26]:
tabla_final.head()

,cedula,mes_campaña,acep_oferta_prev,mes_ult_act,mes_ult_camp,num_act_utl_meses,num_camp_ult_meses,num_meses_ult_actv,num_meses_ult_camp,num_no_aceptado,data_camp
66850,3709,2019-12-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,201912
60661,3876,2019-11-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,201911
20046,4444,2019-05-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,201905
28159,8205,2019-06-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,201906
27802,15237,2019-06-01,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,201906


In [27]:
buro_mdt.head()

,tipo_id,tipo_id_homologado,no_identificacion,id_cliente,fecha_buro,numero_obligaciones_activasdif,porcentaje_utilizacion,quanto_mod,valor_cuotas_codeudores_smlv,valor_utilizado_smlv
735871,1,1000003,206,421952294130855802,202101,0.000000,0.0000,3669000.0,0.0,0.000000
871475,1,1000003,357,702552294130987801,202101,-0.166667,2.1094,3432000.0,0.0,1.973526
631622,1,1000003,1112,543052294130923801,202101,-0.333333,0.0000,1676000.0,0.0,0.000000
656626,1,1000003,1272,962752294130988601,202101,0.142857,0.7924,18188000.0,0.0,66.737771
713245,1,1000003,1898,598952294131470701,202101,0.000000,0.0000,2782000.0,0.0,0.000000


In [28]:
pasvivos_df.head()

,tipo_id,identificacion,fecha_pasivo,pasv_antig_total,pasv_num_meses_ult_apertura,pasv_dias_desde_ultima_trans,pasv_saldo_total_1mes_atras,pasv_saldo_ca_1mes_atras
0,1,946,202102,37.533333,37.533333,54.0,4.822799,4.822799
1,1,946,202103,38.466667,38.466667,13.0,7.638230,7.638230
2,1,946,202104,39.500000,39.500000,44.0,5.221767,5.221767
3,1,1131,202102,1.866667,1.866667,NaN,0.383694,0.000000
4,1,1131,202103,2.800000,2.800000,NaN,0.767388,0.000000


In [29]:
poblacion_total.head()

,id_cliente,fecha_camp,id_numero_cliente,id_tp_cd,periodo,data_camp,fecha_buro,fecha_activos,fecha_pasivo,fecha_lib,fecha_tx_pav
0,102652297589965301,202101,27079010,1000003,04-2021,202105,202101,202104,202104,202104,202101
1,102652296060855001,202101,14972564,1000003,04-2021,202105,202101,202104,202104,202104,202101
2,102652304429247101,202101,80354687,1000003,04-2021,202105,202101,202104,202104,202104,202101
3,102652295542372001,202101,11222544,1000003,04-2021,202105,202101,202104,202104,202104,202101
4,102652297598623801,202101,27076024,1000003,04-2021,202105,202101,202104,202104,202104,202101


## Process Filtering

In [22]:
import numpy as np
import pandas as pd
pd.set_option('max_columns',None)
pd.set_option('max_rows',None)

import s3fs
import pyarrow as pa
import pyarrow.parquet as pq

def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

fs = s3fs.S3FileSystem()

"""
MODULE: processing_filtering
This script extracts activos information from the activos file
Steps:
1. Get activos file
2. Get the needed columns and rows from activos file including the needed historical period
3. Create additional columns
"""

def processing_filtering(input_paths_activos):
    """Pending docstring"""
    # 1. Get filter file
    ids_final = get_filtering_file(input_paths_activos)
    print('get_filtering_file')
    # 2. Get demog file
    data_demog = get_demog_file(input_paths_activos)
    print('get_demog_file')
    # 3. Join between 
    ids_final2 = pd.merge(ids_final,data_demog[['id_cliente','id_numero_cliente','id_tp_cd']], on='id_cliente', how='left')
    print('ids_final_df')
    print('filtering_df created successfully')

    return ids_final2


def get_filtering_file(input_path):
    """ Gets the input_path to the activos file, drops some not useful columns
     and outputs a DataFrame
    :param input_path to activos file location
    :return: DataFrame
    """
    
    mes = '01'  ### REVISAR!!!!
    anio = '2021'   ### REVISAR!!!!
    
    filtering_file_path = input_path['libranzas']
    filtering = spark_read_parquet(filtering_file_path)

    filtering.rename(columns=lambda x: x.lower(), inplace=True)
    
    # Filter and casting variables
    filtering = filtering[filtering['id_cliente'].notnull()]
    filtering['id_cliente'] = filtering['id_cliente'].astype('int')
    
    ids = filtering.loc[~filtering['ds_estado_actual'].isin(['Cancelada','Castigado']),'id_cliente'].unique()
    ids_final = pd.DataFrame({'id_cliente':ids})
    ids_final['fecha_camp'] = int(anio+mes)  ### REVISAR!!!!

    return ids_final


def get_demog_file(input_path):
    """ Gets the input_path to the activos file, drops some not useful columns
     and outputs a DataFrame
    :param input_path to activos file location
    :return: DataFrame
    """

    # read the new table
    data_demog = spark_read_parquet(input_path['clientes'])
    data_demog = data_demog[data_demog['ref_num'].str.isdigit()]

    # Join cedula, typeid and casting variables
    data_demog = data_demog.rename(columns={'cont_id':'id_cliente',
                                       'ref_num':'id_numero_cliente'})
    data_demog['id_tp_cd'] = data_demog['id_tp_cd'].astype('str')
    
    # Drop duplicates and casting variables
    data_demog = data_demog.drop_duplicates(subset=['id_tp_cd', 'id_numero_cliente'], keep='first')
    data_demog['id_cliente'] = data_demog['id_cliente'].fillna('-99').astype(np.int64)

    return data_demog

In [23]:
s3fs.__version__

'0.6.0'

In [8]:
ids_final2 = processing_filtering(input_paths_activos)

print(ids_final2.shape)
ids_final2.head()

## Process Activos

In [9]:
import pandas as pd 
import numpy as np 

pd.set_option('max_columns',None)
pd.set_option('max_rows',None)

from datetime import datetime, timedelta 
from dateutil.relativedelta import relativedelta
import gc

import s3fs
import pyarrow as pa
import pyarrow.parquet as pq

"""
MODULE: processing_activos
This script extracts activos information from the activos file
Steps:
1. Get activos file
2. Get the needed columns and rows from activos file including the needed historical period
3. Create additional columns
"""


def process_activos(input_paths_activos):
    """Pending docstring"""
    # 1. Get activos file
    activos = get_activos_file(input_paths_activos)
    print('get_activos_file')
    # 2. Deletes duplicates and casts variables
    activos = get_activos_info(activos)
    print('get_activo_info')
    # 3. Creates aux variables requeried for the final explanatory variables
    activos = create_activos_info(activos)
    print('create_activos_info')
    # 4. Aggregate and produce client-level output data frame
    activos_cli_fec = get_activos_cliente(activos)
    print('get_activos_cliente')
    # 5. Produce client-level output data frame with explanatory variables
    activos_final = create_activos_df(activos, activos_cli_fec)
    print('create_activos_df')
    print('activos_df created successfully')

    return activos_final


def get_activos_file(input_path):
    """ Gets the input_path to the activos file, drops some not useful columns
     and outputs a DataFrame
    :param input_path to activos file location
    :return: DataFrame
    """

    activos_file_path = input_path['activo']
    activos = spark_read_parquet(activos_file_path)

    activos.rename(columns=lambda x: x.lower(), inplace=True)

    if activos['nroident'].isnull().sum() > 0:
        activos = activos.loc[activos['nroident'].notnull()]

    str_date = activos_file_path[0][-6:]
    activos['fecha_activos'] = pd.to_datetime(str_date, format='%Y%m').strftime('%Y%m')

    # cast nroident
    activos['nroident'] = activos['nroident'].fillna('-999').astype(np.int64)
    activos['tipo_ident_hom'] = activos['tipo_ident_hom'].fillna('-999').astype(np.int64)
    activos = activos[
        ['nroident','tipo_ident_hom', 'id_cliente', 'nrooblig', 'fecha_activos', 'dias_morak', 'lineacredi', 'producto', 'sal_capita',
         'vlr_cuota', 'vlr_desemb']]

    return activos


def get_activos_info(activos):
    """Deletes duplicates and casts variables
    :param activos to activos file location
    :return: DataFrame
    """

    # delete negative nroident
    activos = activos[(activos['nroident'] > 0)]

    # delete duplicates
    activos = activos.sort_values(['nroident','tipo_ident_hom', 'fecha_activos'])
    activos = activos.drop_duplicates(subset=['nroident','tipo_ident_hom', 'fecha_activos'], keep='first')

    # casting variables
    activos['fecha_activos'] = activos['fecha_activos'].astype(int)

    return activos


def create_activos_info(activos):
    """Creates aux variables requeried for the final explanatory variables
    :param activos to activos file location
    :return: DataFrame
    """
    # Creates salario minimo mensual vigente
    activos['ano_po'] = activos['fecha_activos'].astype(str).str[0:4].astype(int)
    activos['smlv'] = np.where(activos['ano_po'] == 2018, 781242,
                               np.where(activos['ano_po'] == 2019, 828116,
                                        np.where(activos['ano_po'] == 2020, 877803,
                                                 np.where(activos['ano_po'] == 2021, 908526,
                                                 np.nan))))

    # Creates active credits flag
    #activos['cuenta_vigente'] = np.where(activos['estado'] == 'ACTIVO', 1, 0)
    activos.loc[:, 'cuenta_vigente'] = 1

    # Creates tipo_prod
    activos['tipo_prod'] = np.where((activos['lineacredi'] == 61) & (activos['producto'] == 'TC'), 'TC_T',
                                    np.where((activos['lineacredi'] == 62) & (activos['producto'] == 'TC'), 'TC_X',
                                             activos['producto']))
   
    #Variable amortizacion a nivel obligación
    activos['amortizacion'] = np.where((activos['vlr_desemb']> 0) & (activos['sal_capita'] >= 0), (activos['vlr_desemb']-activos['sal_capita'])/activos['vlr_desemb'],
                                       np.where((activos['vlr_desemb'] > 0) & np.isnan(activos['sal_capita']),1,np.nan))

    return activos


def get_activos_cliente(activos):
    """Takes a product-level data frame and returns a client-level data frame,
    :param activos complete activos DataFrame
    :return: DataFrame, client-level
    """

    # Creates data base at 'nroident','fecha_activos' level.
    activos_cli_fec = pd.DataFrame(activos.groupby(['nroident','tipo_ident_hom', 'fecha_activos'])[['smlv']].max())

    return activos_cli_fec


def create_activos_df(activos, activos_cli_fec):
    """Creates client-level final variables
    :param activos complete activos DataFrame
    :param activos_cli_fec containing activos relevant rows and columns and activos_cli_fec client level data base
    :return: DataFrame
    """

    # Numero de obligaciones libranza al dia
    activos_cli_fec['amortizacion_min_lb'] = activos[
        (activos['cuenta_vigente'] == 1) & (activos['producto'] == 'LB')].groupby(
        ['nroident','tipo_ident_hom','fecha_activos'])[['amortizacion']].min()


    # Saldo y cuota
    activos_cli_fec['cuota_max_lb']  = activos[
        (activos['producto'] == 'LB') & (activos['cuenta_vigente'] == 1)].groupby(
        ['nroident','tipo_ident_hom','fecha_activos'])[['vlr_cuota']].max()

    # Saldo y cuota / smlv
    activos_cli_fec['cuota_max_lb_smlv'] = activos_cli_fec['cuota_max_lb'] / activos_cli_fec['smlv']

    activos_cli_fec = activos_cli_fec.reset_index()
    activos_out = activos_cli_fec[
        ['nroident','tipo_ident_hom', 'fecha_activos', 'amortizacion_min_lb', 'cuota_max_lb_smlv']]
    print('Shape inicial:',activos_out.shape[0])
    activos_out = activos_out.loc[activos_out['amortizacion_min_lb'].notnull()]
    print('Shape final:',activos_out.shape[0])
   
    return activos_out

def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

In [10]:
# data_activos = process_activos(input_paths_activos)
# print(data_activos.shape)
# data_activos.head()

## Process Campañas Activos

In [11]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pandas_profiling
import os
from datetime import datetime, timedelta
import gc
from datetime import datetime
from dateutil.relativedelta import relativedelta

import boto3
import boto3.session

pd.set_option('display.max_columns',None)

import s3fs
import pyarrow.parquet as pq


def process_camp_activos(input_paths_activos):
    """Pending docstring"""
    
    # Pre_crm. from different route
    data_crm1 = get_crm_new_file(input_paths_activos)
    print('get_crm_new_file')
    # Pre_vend. from different route
    data_vend1 = get_vend_new_file(input_paths_activos)
    print('get_vend_new_file')
    
    # 1. Get crm file
    data_crm = get_crm_file(input_paths_activos)
    print('get_crm_file')
    # 2. Deletes duplicates and casts variables
    data_crm = get_crm_info(data_crm, data_crm1)
    print('get_crm_info')
    print(data_crm.head())
    
    # 3. Get vend file
    data_vend = get_vend_file(input_paths_activos)
    print('get_vend_file')
    # 4. Deletes duplicates, casts variables and union data_crm - data_vend
    data_camp = get_vend_info(data_vend, data_crm, data_vend1)
    print('get_vend_info')
    print(data_camp.head())
    
    # 5. Get activaciones file
    data_act1 = get_activaciones_file(input_paths_activos)
    print('get_activaciones_file')
    # 6. Get activaciones_diarias file
    data_act = get_acti_diarias_file(input_paths_activos)
    print('get_acti_diarias_file')
    # 7. Deletes duplicates and casts variables
    data_act = get_act_diarias_info(data_act, data_act1)
    print('get_act_diarias_info')
    
    # 8. Creates aux variables requeried for the final explanatory variables
    base_final = create_func_activa(data_camp, data_act)
    print('create_func_activa')
    # 9. Deletes duplicates and casts variables
    base_final = get_func_activa_info(base_final)
    print('get_func_activa_info')
    
    # 10. Creates aux variables requeried for the final explanatory variables
    tabla_final = create_func_var_camp_act(base_final)
    print('create_func_var_camp_act')
    # 11. Deletes duplicates and casts variables
    tabla_final = get_func_var_camp_act(tabla_final)
    print('get_func_var_camp_act')
    print('camp_activos_df created successfully')  
    
    return tabla_final

#########################################################
################### Bases de campañas ###################
#########################################################

#####Prep. crm_new
def get_crm_new_file(input_paths):
    
    path_file = get_s3_parquet_flag(input_paths['canpañas'], 0)[-1]
    data_temp = pd.read_csv(path_file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str})
    data_temp = data_temp.sort_values(['base_name','date','clientid'])
    data_temp = data_temp.groupby(['clientid','base_name']).first()
    data_temp.reset_index(inplace=True)
    
    ## Filtrar express-la14
    data_temp = data_temp.loc[~data_temp['base_name'].str.contains('EXPRES_LA14')]
    data_temp = data_temp.rename(columns={'clientid':'NUM_DOC'})
    
    ## Oficinas
    data_crm = data_temp.loc[~data_temp['base_name'].str.contains('OFICINA_BOGOTA')]
    ## tipo campaña
    data_crm['TIPO CAMPAÑA'] = 'Preaprobados'
    data_crm['tipo_camp'] = 'of-preaprobados'
    # Casting variables
    data_crm['FECHA'] = pd.to_datetime(data_crm['date'],format='%Y-%m-%d')
    data_crm['mes_campaña'] = pd.to_datetime(np.where(data_crm['FECHA'].dt.day>=28, 
                                                      data_crm['FECHA']+timedelta(days=10),
                                                      data_crm['FECHA']),format='%Y-%m-%d')
    data_crm['mes_campaña2'] = pd.to_datetime(data_crm['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_crm['mes_campaña'] = pd.to_datetime(data_crm['mes_campaña'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_crm1 = data_crm.copy()
    
    return data_crm1


#####Prep. vend_new
def get_vend_new_file(input_paths):
    
    path_file = get_s3_parquet_flag(input_paths['canpañas'], 0)[-1]
    data_temp = pd.read_csv(path_file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str})
    data_temp = data_temp.sort_values(['base_name','date','clientid'])
    data_temp = data_temp.groupby(['clientid','base_name']).first()
    data_temp.reset_index(inplace=True)
    
    ## Filtrar express-la14
    data_temp = data_temp.loc[~data_temp['base_name'].str.contains('EXPRES_LA14')]
    data_temp = data_temp.rename(columns={'clientid':'NUM_DOC'})
    
    data_vend = data_temp.loc[~data_temp['base_name'].str.contains('CALL_ATENTO')]
    # Casting variables
    data_vend['TIPO CAMPAÑA'] = 'Preaprobados'
    data_vend['tipo_camp'] = 'at-preaprobados'
    data_vend['FECHA'] = pd.to_datetime(data_vend['date'],format='%Y-%m-%d')
    data_vend['mes_campaña'] = pd.to_datetime(data_vend['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_vend1 = data_vend.copy()
    
    return data_vend1


#####1. OFICINAS

def get_crm_file(input_paths):
    files = input_paths['crm']
    
    for ix, file in enumerate(files):
        print('File:', file)
        
        data_temp = pd.read_csv(file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str}) 
        
        if ix == 0:
            data_crm = data_temp
        else:
            data_crm = pd.concat([data_crm,data_temp], ignore_index=True)
            
        del data_temp
    
    return data_crm

## 2.    
## tipo campaña
def get_crm_info(data_crm, data_crm1):
    
    data_crm['TIPO CAMPAÑA'] = 'Preaprobados'
    data_crm['tipo_camp'] = 'of-preaprobados'
    
    # Casting variables
    data_crm['FECHA'] = pd.to_datetime(data_crm['FECHA'],format='%d/%m/%Y')
    data_crm['mes_campaña'] = pd.to_datetime(np.where(data_crm['FECHA'].dt.day>=28,
                                                      data_crm['FECHA']+timedelta(days=10), 
                                                      data_crm['FECHA']),format='%Y-%m-%d')
    data_crm['mes_campaña2'] = pd.to_datetime(data_crm['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_crm['mes_campaña'] = pd.to_datetime(data_crm['mes_campaña'].dt.strftime('%Y-%m'),format='%Y-%m')
    
    data_crm = pd.concat([data_crm1, data_crm], ignore_index=True)
    
    return data_crm


##### 3. ATENTO
def get_vend_file(input_paths):

    files = input_paths['vendors']
    
    for ix, file in enumerate(files):
        print('File:', file)
        
        data_temp = pd.read_csv(file, sep='|', encoding='ISO8859-1',quoting=1, dtype={'ID_CLIENTE':str,'NUM_DOC':str})
        if ix == 1:
            data_temp['FECHA2'] = pd.to_datetime(data_temp['FECHA'],format='%d/%m/%Y')
            data_temp = data_temp[data_temp['FECHA2']<'2020-12-01']
            data_temp = data_temp[['TIP_DOC','NUM_DOC','ID_CLIENTE',' CUPO_APROBADO ','FECHA','TIPO CAMPAÑA']]
        if ix == 0:
            data_vend = data_temp
        else:
            data_vend = pd.concat([data_vend,data_temp], ignore_index=True)
            
        del data_temp
    
    return data_vend


#4. Casting variables
def get_vend_info(data_vend, data_crm, data_vend1):
    
    data_vend['tipo_camp'] = np.where(data_vend['TIPO CAMPAÑA'] == 'Preaprobados','at-preaprobados','at-perfilados')
    data_vend['FECHA'] = pd.to_datetime(data_vend['FECHA'],format='%d/%m/%Y')
    data_vend['mes_campaña'] = pd.to_datetime(data_vend['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    
    data_vend = pd.concat([data_vend1,data_vend], ignore_index=True)
    
    cols = ['NUM_DOC', 'FECHA', 'mes_campaña', 'TIPO CAMPAÑA', 'tipo_camp']
    
    data_vend.columns = data_vend.columns.str.strip()
    data_camp = pd.concat([data_crm[cols], data_vend[cols]], ignore_index=True)[cols]
        
    data_camp['NUM_DOC'] = data_camp['NUM_DOC'].astype(np.int64)
    data_camp['FECHA'] = pd.to_datetime(data_camp['FECHA'])
    data_camp['mes_campaña'] = pd.to_datetime(data_camp['mes_campaña'])
    data_camp = data_camp.rename(columns={'NUM_DOC':'cedula'})
    
    return data_camp

    
##5. ACTIVACIONES
def get_activaciones_file(input_paths):

    data_act2 = pd.read_csv(input_paths['activaciones'][0], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
    data_act2 = data_act2.drop_duplicates()
    print(data_act2.shape)
    
    data_act1 = pd.read_csv(input_paths['activaciones'][1], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
    data_act1 = data_act1.drop_duplicates()
    print(data_act1.shape)
    
    # Uniendo bases
    data_act2['fecha_activacion'] = data_act2['fecha_activacion'].str.replace('/20','/2020')
    data_act1 = pd.concat([data_act2,data_act1], ignore_index=True)
    
    # Casting variables
    data_act1['fecha_activacion'] = pd.to_datetime(data_act1['fecha_activacion'],format='%d/%m/%Y')
    data_act1['mes_activacion'] = pd.to_datetime(data_act1['fecha_activacion'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_act1.columns = data_act1.columns.str.lower()
    data_act1 = data_act1.rename(columns={'identificacion':'cedula'})
    del data_act2
    
    return data_act1


##6. bases diarias
def get_acti_diarias_file(input_paths):

    files = get_s3_parquets(input_paths['acti_diarias'][0])
    
    for ix, file in enumerate(files):
        print('File:', file)
        data_temp = spark_read_parquet(file)
        if ix == 0:
            data_act = data_temp
        else:
            data_act = pd.concat([data_act,data_temp], ignore_index=True)
            
        del data_temp
        
    return data_act

        
#7. Casting variables     
def get_act_diarias_info(data_act, data_act1):

    data_act['fecha_activacion_plastico'] = pd.to_datetime(data_act['fecha_activacion_plastico'])
    data_act['mes_activacion'] = pd.to_datetime(data_act['fecha_activacion_plastico'].dt.strftime('%Y-%m'),format='%Y-%m')
    
    ## Renombrando variables
    data_act = data_act.rename(columns={'id_cli':'cedula',
                                       'tipo_id_cliente':'tipo_identificacion',
                                       'fecha_activacion_plastico':'fecha_activacion'})
    
    data_act = data_act[['numero_cuenta', 'numero_tarjeta', 'cedula', 'tipo_identificacion', 'fecha_activacion', 'id_cliente',
           'mes_activacion']]
    
    ## Uniendo bases
    data_act = pd.concat([data_act1, data_act], ignore_index=True)
    data_act = data_act.drop_duplicates()

    del data_act1

    ## Cast variables
    data_act['id_cliente'] = data_act['id_cliente'].fillna('-9999').astype(np.int64)
    data_act['tipo_identificacion'] = data_act['tipo_identificacion'].fillna('-9999').astype(np.int64)
    data_act['cedula'] = data_act['cedula'].astype(np.int64)

    return data_act


###8. func_activa
def create_func_activa(data_camp, data_act):

    str_dates = ['2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
                 '2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10', 
                 '2020-11','2020-12','2021-01','2021-02','2021-03','2021-04','2021-05','2021-06']
    n_mes = 3 # numero de meses
    
    for ix, date in enumerate(str_dates):
        print('Fecha:',date)
        date = pd.to_datetime(date,format='%Y-%m')
        nom_col = 'camp_'+date.strftime('%Y%m')
        
        base = data_camp[data_camp['mes_campaña']==date]
    
        if ix == 0:
            base_final = func_activa(base, data_act, date, n_mes, nom_col)
            del base
        else:
            base_temp = func_activa(base, data_act, date, n_mes, nom_col)
            base_final = pd.concat([base_final, base_temp], ignore_index=True)
            
            del base, base_temp

    return base_final


#9. Casting variables
def get_func_activa_info(base_final):
    
    base_final = base_final[base_final['cedula'].notnull()]
    
    # base_final.rename(columns={' CUPO_APROBADO ':'CUPO_APROBADO'}, inplace=True)
    # base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].str.strip()
    # base_final['CUPO_APROBADO'] = [str(x).replace('.','') for x in base_final['CUPO_APROBADO']]
    # base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].astype('float')
    
    cols = ['camp_201901','camp_201902','camp_201903','camp_201904','camp_201905','camp_201906','camp_201907',
            'camp_201908','camp_201909','camp_201910','camp_201911','camp_201912','camp_202001','camp_202002',
            'camp_202003','camp_202004','camp_202005','camp_202006','camp_202008','camp_202009','camp_202010',
            'camp_202011','camp_202012','camp_202101','camp_202102','camp_202103','camp_202104','camp_202105',
            'camp_202106','venta']
    
    base_final.loc[:,cols] = base_final[cols].fillna(0)
    
    # Casting variables
    base_final['fecha_activacion'] = pd.to_datetime(base_final['fecha_activacion'], format='%Y-%m%d')
    base_final['mes_activacion'] = pd.to_datetime(base_final['mes_activacion'], format='%Y-%m%d')

    return base_final

##10 Crear variable meses
def create_func_var_camp_act(base_final):

    str_dates = ['2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
                 '2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09',
                 '2020-10','2020-11','2020-12','2021-01','2021-02','2021-03','2021-04','2021-05','2021-06']
    
    for ix, dates in enumerate(str_dates):
        print('Periodo:', dates)
        date = pd.to_datetime(dates,format='%Y-%m')
        
        ids = list(base_final.loc[base_final['mes_campaña']==date,'cedula'].unique())
        gp_temp = base_final[base_final['mes_campaña']<date].groupby('cedula')
        # gp_temp_ap = base_final[(base_final['mes_campaña']<date)&(base_final['CUPO_APROBADO'].notnull())].groupby('cedula')
        
        if ix == 0:
            tabla_final = func_var_camp_act(gp_temp, ids, date)
            
        #    if len(gp_temp_ap)>0:
        #        tabla_final['prom_cupo_aprob'] = gp_temp_ap['CUPO_APROBADO'].mean()
        
            tabla_final.index.rename('cedula', inplace=True)
            tabla_final.reset_index(inplace = True)
        else:
            base_temp = func_var_camp_act(gp_temp, ids, date)
            
        #    if len(gp_temp_ap)>0:
        #        base_temp['prom_cupo_aprob'] = gp_temp_ap['CUPO_APROBADO'].mean()
        
            base_temp.index.rename('cedula', inplace=True)
            base_temp.reset_index(inplace = True)
            
            tabla_final = pd.concat([tabla_final,base_temp], ignore_index=True)
            
            del base_temp
        del gp_temp, date
    
    return tabla_final

    
##11. Cast index
def get_func_var_camp_act(tabla_final):

    tabla_final['cedula'] = tabla_final['cedula'].fillna('-999')
    tabla_final['cedula'] = tabla_final['cedula'].astype('int')
    
    tabla_final= tabla_final[['cedula', 'mes_campaña','acep_oferta_prev', 'mes_ult_act',
                              'mes_ult_camp', 'num_act_utl_meses', 'num_camp_ult_meses',
                              'num_meses_ult_actv', 'num_meses_ult_camp', 'num_no_aceptado']]
    
    tabla_final = tabla_final[(tabla_final['cedula'].notnull()) & (tabla_final['mes_campaña'].notnull())]
    tabla_final = tabla_final.drop_duplicates()
    
    tabla_final['data_camp'] = pd.to_datetime(tabla_final['mes_campaña'], format='%Y-%m-%d').dt.strftime('%Y%m').astype('int')
    tabla_final = tabla_final.sort_values(['cedula', 'mes_campaña'])
    
    return tabla_final


######## Funciones utilizadas
    
def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

def func_activa(base, data_act, date, n_mes, nom_col):
    if len(base)>0:
        date_fin = date + relativedelta(months=n_mes)

        data_act_temp = data_act.loc[(data_act['mes_activacion']>=date) & (data_act['mes_activacion']<=date_fin),['cedula','Numero_cuenta','fecha_activacion','mes_activacion']].drop_duplicates()
        ids_mes = data_act_temp['cedula'].unique()
        
        base.loc[base['cedula'].isin(ids_mes),'venta'] = 1    
        base.loc[base['venta'].isnull(),'venta'] = 0
        
        base.loc[:,nom_col] = 1   
        #base = pd.merge(base,data_act_temp,on=['cedula','ID_CLIENTE'],how='left')
        base = pd.merge(base,data_act_temp,on=['cedula'],how='left')
    
    return base

def func_var_camp_act(gp_temp, ids, date):
    
    #producing the first dataframe
    base_out = pd.DataFrame(index=ids) # this is different from the actual prod code
    base_out['mes_campaña'] = date.strftime('%Y-%m-%d')
    
    ## Calculando días primer uso
    base_out['mes_ult_camp'] =  gp_temp['mes_campaña'].max()
    base_out['mes_ult_act'] =  gp_temp['fecha_activacion'].max()
    base_out.loc[base_out['mes_ult_camp'].notnull(),'num_meses_ult_camp'] = (date - base_out.loc[base_out['mes_ult_camp'].notnull(),'mes_ult_camp'])/pd.offsets.Day(30)
    base_out.loc[base_out['mes_ult_act'].notnull(),'num_meses_ult_actv'] = (date - base_out.loc[base_out['mes_ult_act'].notnull(),'mes_ult_act'])/pd.offsets.Day(30)
    
    base_out['num_camp_ult_meses'] = gp_temp['mes_campaña'].count()
    base_out['num_act_utl_meses'] = gp_temp['venta'].sum()
    base_out['num_no_aceptado'] = gp_temp['venta'].value_counts().unstack()[0].fillna(0).astype(int)
    base_out['acep_oferta_prev'] = gp_temp['venta'].max()
    
    return base_out

def _key_existing_size__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    for obj in response.get("Contents", []):
        if obj["Key"] == key:
            return obj["Size"]
        
def _key_files__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    return response.get("Contents", [])

def parquet_exists(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)+ "/_SUCCESS"
    size = _key_existing_size__list(s3, bucket, key)
    if size is not None:
        print("File exists:", s3_url)
        return True
    return False

def get_s3_parquets(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    objects = _key_files__list(s3, bucket, key)
    files = ["s3://" + bucket + "/" + file["Key"][:-9] for file in objects if file["Key"].split("/")[-1]=="_SUCCESS"]
    return files

def get_s3_parquet_flag(s3_url, flag):
    """
    s3_url: path donde se desea buscar
    flag: permite identificar por cual camino entrar: 1: buscar archivo especifico, 0: buscar carpeta, ejemplo las std
    """
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    print('bucket_last_file: ', bucket)
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    print('key_last_file: ', key)
    files = _key_files__list(s3, bucket, key)
    files = ["s3://" + bucket + "/" + file["Key"] for file in files]
    return files

In [12]:
# tabla_final = process_camp_activos(input_paths_activos)
# print(tabla_final.shape)
# tabla_final.head()

## Proccess Buro

In [13]:
"""
MODULE: processing_buro
This script extracts Buró information from the Buró file
Steps:
1. Get buro files
2. Get the needed columns and rows from Buro file including the needed historical period
3. Create additional columns
"""

import numpy as np
import pandas as pd
import s3fs
import pyarrow.parquet as pq

def process_buro(input_paths_activos):
    """Using the input_paths for the last two buro files, this function puts everything together"""

    # 1. Gets and appends the last two buro files
    buro = get_buro_file(input_paths_activos)
    print('get_buro_file')
    # 2. Get the needed columns and rows from buro file including the needed historical period
    buro = get_buro_info(buro)
    print('get_buro_info')
    # 3. Perform simple imputation to output data frame
    buro_final = create_buro_df(buro)
    print('create_buro_df')
    print('buro_df created successfully')

    return buro_final


def get_buro_file(input_path):
    """ Gets the input_paths of the last two the buro files, appends them and drops some not useful columns
     and outputs a DataFrame
    :param
    input_path1: location of the oldest buro file
    input path2: location of the most recent buro file
    :return: DataFrame
    """
    
    input_path2 = get_s3_parquets(input_path['buro'])[-2]
    
    buro_1 = spark_read_parquet(input_path2)
    buro_1.columns = buro_1.columns.str.lower()
    buro_1 = buro_1.rename(columns={'cont_id': 'id_cliente'})

    buro_1['marca_buro'] = 1
    if buro_1['no_identificacion'].isnull().sum() > 0:
        buro_1 = buro_1.loc[buro_1['no_identificacion'].notnull()]

    buro_1['no_identificacion'] = buro_1['no_identificacion'].astype('str')
    buro_1['no_identificacion'] = buro_1['no_identificacion'].astype(np.int64)

    input_path1 = get_s3_parquets(input_path['buro'])[-1]
    buro_2 = spark_read_parquet(input_path1)
    buro_2.columns = buro_2.columns.str.lower()
    buro_2 = buro_2.rename(columns={'cont_id': 'id_cliente'})

    buro_2['marca_buro'] = 2
    if buro_2['no_identificacion'].isnull().sum() > 0:
        buro_2 = buro_2.loc[buro_2['no_identificacion'].notnull()]

    buro_2['no_identificacion'] = buro_2['no_identificacion'].astype('str')
    buro_2['no_identificacion'] = buro_2['no_identificacion'].astype(np.int64)

    buro = pd.concat([buro_1,buro_2], ignore_index=True)
    
    if buro['no_identificacion'].isnull().sum() > 0:
        buro = buro.loc[buro['no_identificacion'].notnull()]

    buro['no_identificacion'] = buro['no_identificacion'].astype(np.int64)
    buro['id_cliente'] = buro['id_cliente'].fillna('-999').astype(np.int64)
    buro = buro[['tipo_id','tipo_id_homologado','no_identificacion', 'fecha_envio',
                 'id_cliente', 'marca_buro', 
                 'numero_obligaciones_activas',
                 'porcentaje_utilizacion',
                 'quanto_mod',
                 'valor_cupos',
                 'valor_utilizado',
                 'valor_cuotas_codeudores']]
    
    buro = buro.fillna(0)

    return buro


def get_buro_info(buro):
    """Deletes duplicates and casts dates
    :param buro to buro file location
    :return: DataFrame
    """
    # casting dates
    buro = buro[buro['fecha_envio'].notnull()]
    buro['fecha_envio'] = buro['fecha_envio'].astype('str')
    buro['fecha_buro'] = (
                buro['fecha_envio'].str.replace('-', '').str[0:4] + buro['fecha_envio'].str.replace('-', '').str[
                                                                    4:6]).astype('int')

    # deleting duplicates
    buro = buro.sort_values(['no_identificacion', 'fecha_buro'])
    buro = buro.drop_duplicates(subset=['no_identificacion', 'fecha_buro'], keep='first')

    return buro


def create_buro_df(buro):
    """Creates client-level variables final variables
    :param buro containing buro relevant rows and columns
    :return: DataFrame
    """
    # creating lag variables
    buro['llave_lag'] = np.where(buro['no_identificacion'] == buro['no_identificacion'].shift(1), 1, 0)

    # variable cupo mercado
    buro['ano_po'] = buro['fecha_buro'].astype(str).str[0:4].astype(int)
    buro['smlv'] = np.where(buro['ano_po'] == 2018, 781242,
                            np.where(buro['ano_po'] == 2019, 828116,
                                     np.where(buro['ano_po'] == 2020, 877803,
                                              np.where(buro['ano_po'] == 2021, 908526,
                                              np.nan))))
    
    if (buro['porcentaje_utilizacion'] > 0).sum()/buro.shape[0] > 0.9:
    
        buro['porcentaje_utilizacion'] = [float(str(x).replace('"','').replace(',','.')) for x in buro['porcentaje_utilizacion']]
        buro['porcentaje_utilizacion'] = buro['porcentaje_utilizacion'].astype('float')
    else:
        buro['porcentaje_utilizacion'] = np.where(buro['valor_cupos']>0, round(buro['valor_utilizado']/buro['valor_cupos'],4),0) 


    buro['valor_cuotas_codeudores_smlv']= buro['valor_cuotas_codeudores']/buro['smlv']
    buro['valor_utilizado_smlv']= buro['valor_utilizado']/buro['smlv']
    
    var = 'numero_obligaciones_activas'
 
    # creating lag variables lag
    buro['llave_lag'] = np.where(buro['no_identificacion'] == buro['no_identificacion'].shift(1), 1, 0)

    buro[var+'lag1']=np.where(buro['llave_lag']==1,buro[var].shift(1),np.nan)
    buro[var+'dif']=np.where( ((np.isnan(buro[var])) | (buro[var]==0)) & ((np.isnan(buro[var+'lag1'])) | (buro[var+'lag1']==0)) ,0,
                              np.where( ((np.isnan(buro[var])) | (buro[var]==0)) & ((buro[var+'lag1']>0)) ,-1,
                              np.where( ((np.isnan(buro[var+'lag1'])) | (buro[var+'lag1']==0)) & ((buro[var]>0)) ,1,
                              np.where( (buro[var+'lag1']>0) & (buro[var]>0) , (buro[var]-buro[var+'lag1'])/buro[var+'lag1'],99))))
    
    del buro[var+'lag1'], var
    
    #Last filter
    buro = buro[buro['marca_buro'] == 2]
    
    buro_out = buro[['tipo_id','tipo_id_homologado','no_identificacion','id_cliente', 'fecha_buro', 
                     'numero_obligaciones_activasdif',
                    'porcentaje_utilizacion',
                    'quanto_mod',
                    'valor_cuotas_codeudores_smlv',
                    'valor_utilizado_smlv']]

    return buro_out

def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

In [14]:
# buro_mdt = process_buro(input_paths_activos)
# print(buro_mdt.shape)
# buro_mdt.head()

## Process Libranzas

In [15]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import s3fs
import pyarrow.parquet as pq
import os
import calendar
import gc

pd.set_option('max_columns',None)
pd.set_option('max_rows',None)

"""
module: processing_libranzas
This script extracts libranzas information from the libranzas file
Steps:
1. Get libranzas file
2. Get the needed columns and rows from libranzas file including the needed historical period
3. Create additional columns
4. Aggregate and produce client-level output data frame
5. Perform simple imputation to output data frame
"""

def process_libranzas(input_paths_activos: str) -> pd.DataFrame:
    """Using the input_path this function puts everything together"""
    # 1. Get libranzas file
    libranzas = get_libranzas_file(input_paths_activos)
    print('get_libranzas_file')
    # 2. Get the needed columns and rows from libranzas file including the needed historical period
    libranzas = get_libranzas_info(libranzas)
    print('get_libranzas_info')
    # 3. Create additional columns
    libranzas = add_columns_to_lib(libranzas)
    print('add_columns_to_lib')
    # 4. Aggregate and produce client-level output data frame
    libranzas = create_lib_df(libranzas)
    print('create_lib_df')
    # 5. Perform simple imputation to output data frame
    libranzas_final = simple_imputation(libranzas)
    
    libranzas_final['periodo'] = int(input_paths_activos['libranzas'][0][-6:])
    libranzas_final.reset_index(inplace=True)
    libranzas_final = libranzas_final.rename(columns={'id_cli':'id_numero_cliente','standard_id':'id_tp_cd'})
    print('libranzas_df created successfully')
    return libranzas_final


def get_libranzas_file(input_path):
    """ Gets the input_path to the libranzas file, drops some not useful columns
     and outputs a DataFrame
    :param input_path to libranzas file location
    :return: DataFrame
    """
    file = input_path['libranzas']
    dataset = pq.ParquetDataset(file, filesystem=fs)
    table = dataset.read()
    lib = table.to_pandas()
    lib.rename(columns=lambda x: x.lower(), inplace=True)
    if lib['id_cli'].isnull().sum() > 0:
        lib = lib.loc[lib['id_cli'].notnull()]

    lib['id_cli'] = lib['id_cli'].astype(np.int64)
    lib['standard_id'] = lib['standard_id'].astype(np.int64)
    
    lib_constant_cols = ['sk_producto_servicio', 'cd_modalidad_pag_int',
                         'ds_modalidad_pag_int', 'cd_periodicidad_pag_int',
                         'ds_periodicidad_pag_int', 'cd_base_liquidacion',
                         'no_obligacion_novada']

    lib.drop(columns=lib_constant_cols, inplace=True)
    
    del table, dataset

    return lib


def get_libranzas_info(lib):
    """Gets libranzas relevant rows and columns
    :param lib to libranzas file location
    :return: DataFrame
    """
    HISTORICAL_YEARS = 13
    # we are working only with "libranzas organicas"
    lib = lib.loc[lib['ds_tipo_libranza'] == 'Organica']

    # eliminating these records, they may have quality issues (less than 1%)
    lib = lib.loc[lib['ds_tipo_credito'].notnull()]

    # necessary date processing
    lib_date_cols = ['fe_solicitud', 'fe_desembolso']
    for col in lib_date_cols:
        lib[col] = pd.to_datetime(lib[col], dayfirst=True, errors='coerce')

    # getting records for the appropriate period
    end_historical_period = pd.Timestamp(get_prev_months_last_date())
    number_of_years = datetime.timedelta(days=int(365.25*HISTORICAL_YEARS))
    beginning_historical_period = end_historical_period - number_of_years
    period_filter = (lib['fe_solicitud'] >= beginning_historical_period) & (lib['fe_solicitud'] <= end_historical_period)
    lib = lib.loc[period_filter]

    # imputing fe_solicitud missing dates (less than 0.5%)
    lib.loc[lib['fe_solicitud'].isnull(), 'fe_solicitud'] = lib.loc[lib['fe_solicitud'].isnull(), 'fe_desembolso']

    # getting only the relevant columns
    lib_relevant_cols = ['id_cli','standard_id', 'sk_rc_libranza', 'fe_solicitud',
                         'vl_monto_solicitado', 'fe_desembolso',
                         'vl_monto_aprobado', 'no_obligacion', 'vl_monto_desembolsado',
                         'no_cuotas', 'vl_total_cuota', 'ds_estado_actual',
                         'vl_tasa', 'ds_tipo_credito', 'ds_tipo_libranza', 'vl_monto_novado']

    lib = lib[lib_relevant_cols]
    lib = lib.loc[~lib['ds_estado_actual'].isin(['Cancelada','Castigado']),:]

    return lib


def add_columns_to_lib(lib):
    """Takes a libranzas DataFrame and adds some useful columns
    :param lib DataFrame containing libranzas relevant rows and columns
    :return: DataFrame
    """
    lib['dias_procesamiento'] = (lib['fe_desembolso'] - lib['fe_solicitud'])/np.timedelta64(1, 'D')
    # this is a necessary fix because there are many dates with fecha de solicitud in 1900
    lib.loc[lib['dias_procesamiento'] > 360, 'dias_procesamiento'] = lib['dias_procesamiento'].median()
    lib.loc[(lib['vl_monto_aprobado'] == 0)|(lib['vl_monto_aprobado'].isnull()),'vl_monto_aprobado'] = lib['vl_monto_desembolsado']

    return lib


def create_lib_df(lib):
    """Creates client-level variables by aggregating the columns
    :param lib containing libranzas relevant rows and columns
    :return: DataFrame
    """
    # producing the first data frame
    gp = lib.groupby(['id_cli','standard_id'])
    lib_out = lib[['id_cli','standard_id']].drop_duplicates()
    lib_out.set_index(['id_cli','standard_id'], inplace=True)

    # number of records for each cliente
    lib_out['num_lib_solicitadas'] = gp.size()

    # these have a very close relation
    lib_out['prom_monto_novado'] = gp['vl_monto_novado'].mean()

    # other averages
    lib_out['prom_n_cuotas'] = gp['no_cuotas'].mean()

    return lib_out


def get_prev_months_last_date():
    """From current date returns the last date of the previous month"""
    current_date = datetime.date.today()
    first_day_of_month = current_date.replace(day=1)
    out = first_day_of_month - datetime.timedelta(days=1)
    return out


def get_prev_prev_months_last_date():
    """From current date returns the last date of the month before the previous month"""
    last_day_of_prev_month = get_prev_months_last_date()
    first_day_of_prev_month = last_day_of_prev_month.replace(day=1)
    out = first_day_of_prev_month - datetime.timedelta(days=1)
    return out


def simple_imputation(df):
    """Gets a data frame and imputes the median for numerical columns and the
    mode (most common category) for categorical columns"""
    numerical_cols = df.select_dtypes(np.number).columns
    categorical_cols = df.select_dtypes(object).columns

    for col in numerical_cols:
        df[col] = df[col].fillna(df[col].median())

    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode().iloc[0])

    return df

In [16]:
# libranzas_df = process_libranzas(input_paths_activos)
# print(libranzas_df.shape)
# libranzas_df.head()

## Process Pasivo

In [17]:
import pandas as pd 
import numpy as np 

pd.set_option('max_columns',None)
#pd.set_option('max_rows',100)

from datetime import datetime, timedelta 
from dateutil.relativedelta import relativedelta

import s3fs
import pyarrow.parquet as pq

fs = s3fs.S3FileSystem()

"""
MODULE: processing_activos
This script extracts activos information from the activos file
Steps:
1. Get activos file
2. Get the needed columns and rows from activos file including the needed historical period
3. Create additional columns
"""


def process_pasivos(input_paths_activos):
    """Pending docstring"""
    # 1. Get activos file
    pasivos = get_pasivos_file(input_paths_activos)
    print('get_pasivos_file')
    
    # 2. Deletes duplicates and casts variables
    pasivos = get_pasivos_info(pasivos)
    print('get_pasivos_info')
     
    # 3. Creates aux variables requeried for the final explanatory variables
    pasivos_last_3 = get_pasivos_last_3(pasivos)
    print('get_pasivos_last_3')
    
    # 4. Aggregate and produce client-level output data frame
    pasivos_last_3 = create_pasivo_last_3_info(pasivos_last_3)
    print('create_pasivo_last_3_info')
    
    # 5. Produce client-level output data frame with explanatory variables
    pasivos_last_3_cli_fec = create_pasivos_df(pasivos_last_3)
    print('create_pasivos_last_3_cli_df')
    
    6. 
    pasv_out = create_pasv_out(pasivos_last_3_cli_fec)
    print('create_pasv_out')
    
    # 7. 
    pasvivos_df = pasivo_last_3_tot(pasivos_last_3_cli_fec, pasv_out)
    print('pasivo_last_3_tot')   
    print('pasivos_df created successfully')

    return pasvivos_df



def get_pasivos_file(input_path):
    """ Gets the input_path to the activos file, drops some not useful columns
     and outputs a DataFrame
    :param input_path to activos file location
    :return: DataFrame
    """

    product_type_dict = {'1':'corriente', '2':'ahorro', '3':'cdt'}
    str_dates = ['202102','202103','202104'] ### vector_fechas
    
    for ix, date in enumerate(str_dates):

        tabla_temp = creando_pasivos_antg(input_path, date)
        tabla_temp.columns = tabla_temp.columns.str.lower()

        ## Cambiar tipo producto:
        tabla_temp['periodo'] = int(date)
        tabla_temp['tipo_producto'] = tabla_temp['cod_producto'].apply(lambda x: str(x)[0]).map(product_type_dict)

        print(tabla_temp.shape)

        print('Periodo:',date)
        if ix == 0:
            tabla_final = tabla_temp
        else:
            tabla_final = pd.concat([tabla_final,tabla_temp], ignore_index=True)

        del tabla_temp

    return tabla_final


def creando_pasivos_antg(input_paths, date):
    
    PASIVOS_MONEY_COLS = ['saldo',
                          'promedio1',
                          'promedio2',
                          'promedio3',
                          'promedio4',
                          'promedio5',
                          'promedio6']

    dict_cc={0:'Nueva',
             1:'Activa',
             2:'Bloqueada',
             3:'Cancelada',
             4:'Inactiva',
             5:'Por Cancelar',
             6:'Saldada',
             7:'Desbloqueada',
             8:'Por Saldar',
             9:'Por Abrir'}

    dict_cdt={1:'Captado',
              2:'Depositado',
              3:'Exigible',
              4:'Cobrado',
              5:'Bloquedo',
              6:'Embargado',
              7:'Retenido',
              8:'Anulado',
              9:'Cancelado',
              10:'Canc. Fraccionamiento',
              11:'Canc. Fusion'}

    dict_ca={0:'Nueva',
             1:'Activa',
             2:'Bloqueada',
             3:'Cancelada',
             4:'Inactiva',
             5:'Por Cancelar',
             6:'Saldada',
             7:'Desbloqueada',
             8:'Por Saldar',
             9:'Por Abrir'}
    
    periodo = input_paths['pasivo_cc'][0][-6:]
    ## 1. CC
    print('Leyendo CC')
    input_cc = input_paths['pasivo_cc']+date
    cc=spark_read_parquet(input_cc)
    cc=cc.rename({'acmcc_scotipide':'tipo_id','id_cli':'identificacion','acmcc_ccu_produ':'cod_producto','acmcc_cfeulttra':'fecha_ultima_transaccion','acmcc_cfeapecue':'fecha_apertura','acmcc_csatotman':'saldo','acmcc_promedmen01':'promedio1','acmcc_promedmen02':'promedio2','acmcc_promedmen03':'promedio3','acmcc_promedmen04':'promedio4','acmcc_promedmen05':'promedio5','acmcc_promedmen06':'promedio6','periodo':'periodo'},axis=1)
    cc['nro_cuenta']=cc['cod_producto'].astype('str')+cc['acmcc_ccu_ofici'].astype('str').str.zfill(3)+cc['acmcc_ccunumcue'].astype('str').str.zfill(5)+cc['acmcc_ccudigver'].astype('str') 
    
    cc['estado']=[dict_cc.get(i) for i in cc['acmcc_ccoestcue'] ]
    cc=cc[(cc['cod_producto']==110)&(cc['estado'].isin(['Nueva','Activa']))].reset_index().drop(['index'],axis=1)
    cc=cc[['tipo_id','identificacion','cod_producto','nro_cuenta','fecha_ultima_transaccion','fecha_apertura','estado','saldo','promedio1','promedio2','promedio3','promedio4','promedio5','promedio6','periodo']]

    ## 2. CDT
    print('Leyendo CDT')
    input_cdt = input_paths['pasivo_cdt']+date
    cdt=spark_read_parquet(input_cdt)
    cdt=cdt.rename({'afmde_scotipide':'tipo_id','id_cli':'identificacion','afmde_fcu_produ':'cod_producto','afmde_ffeultmod':'fecha_ultima_transaccion','afmde_ffe_depos':'fecha_apertura','afmde_fva_depos':'saldo','afmde_promedio':'promedio1','periodo':'periodo'},axis=1)
    cdt['nro_cuenta']=cdt['cod_producto'].astype('str')+cdt['afmde_fcu_ofici'].astype('str').str.zfill(3)+cdt['afmde_fcunumcue'].astype('str').str.zfill(7)+cdt['afmde_fcudigver'].astype('str') 
    cdt['estado']=[dict_cdt.get(i) for i in cdt['afmde_fco_estde'] ]

    cdt['promedio2']=np.nan
    cdt['promedio3']=np.nan
    cdt['promedio4']=np.nan
    cdt['promedio5']=np.nan
    cdt['promedio6']=np.nan

    cdt=cdt[(cdt['estado'].isin(['Captado','Depositado','Exigible']))].reset_index().drop(['index'],axis=1)
    cdt=cdt[['tipo_id','identificacion','cod_producto','nro_cuenta','fecha_ultima_transaccion','fecha_apertura','estado','saldo','promedio1','promedio2','promedio3','promedio4','promedio5','promedio6','periodo']]

    ## 3. CA
    print('Leyendo CA')
    input_ca = input_paths['pasivo_ca']+date
    ca=spark_read_parquet(input_ca)
    ca=ca.rename({'ahmah_scotipide':'tipo_id','id_cli':'identificacion','ahmah_hcu_produ':'cod_producto','ahmah_hfeulttra':'fecha_ultima_transaccion','ahmah_hfeapecue':'fecha_apertura','ahmah_hsatotman':'saldo','ahmah_hvepromen01':'promedio1','ahmah_hvepromen02':'promedio2','ahmah_hvepromen03':'promedio3','ahmah_hvepromen04':'promedio4','ahmah_hvepromen05':'promedio5','ahmah_hvepromen06':'promedio6','periodo':'periodo'},axis=1)
    ca['nro_cuenta']=ca['cod_producto'].astype('str')+ca['ahmah_hcu_ofici'].astype('str').str.zfill(3)+ca['ahmah_hcunumcue'].astype('str').str.zfill(5)+ca['ahmah_hcudigver'].astype('str')  
    ca['estado']=[dict_ca.get(i) for i in ca['ahmah_hcoestcue'] ]

    ca=ca[(ca['estado'].isin(['Nueva','Activa']))].reset_index().drop(['index'],axis=1)
    ca=ca[['tipo_id','identificacion','cod_producto','nro_cuenta','fecha_ultima_transaccion','fecha_apertura','estado','saldo','promedio1','promedio2','promedio3','promedio4','promedio5','promedio6','periodo']]

    ## 4. Apppend
    print('Append archivos')
    total=cc.append(cdt).append(ca).reset_index().drop(['index'],axis=1)
    total['fecha'] = date
    total['fecha'] = pd.to_datetime(total['fecha'], format='%Y%m')
    total['cod_producto']=total['cod_producto'].astype('str')

    total['tipo_ident_hom']=np.where(total['tipo_id']==1,'1000003',
                                       np.where(total['tipo_id']==2,'1000005',
                                               np.where(total['tipo_id']==3,'1000007',
                                                       np.where(total['tipo_id']==4,'1000002',
                                                               np.where(total['tipo_id']==5,'1000004',
                                                                         np.where(total['tipo_id']==6,'1000006',
                                                                                   np.where(total['tipo_id']==7,'1000007',
                                                                                       np.where(total['tipo_id']==8,'1000008',
                                                                                                np.where(total['tipo_id']==9,'1000009','99999')))))))))

    frame_total=total
    
    # Cast
    if frame_total['identificacion'].isnull().sum() > 0:
        frame_total = frame_total.loc[frame_total['id_cli'].notnull()]

    frame_total['identificacion'] = frame_total['identificacion'].astype(np.int64)
    frame_total['tipo_ident_hom'] = frame_total['tipo_ident_hom'].astype(np.int64)
    
    return frame_total


def get_pasivos_info(tabla_final):
    """Deletes duplicates and casts variables
    :param activos to activos file location
    :return: DataFrame
    """

    # Cast dates
    tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'])
    tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'])

    # Reset index  ### REVISAR si quitar o no
    tabla_final.reset_index(inplace=True)
    tabla_final = tabla_final.drop(columns={'index'})

    # Cast variables
    tabla_final = tabla_final[tabla_final['identificacion'].notnull()]
    tabla_final = tabla_final[tabla_final['nro_cuenta'].isna()==False]

    tabla_final['identificacion'] = tabla_final['identificacion'].astype('int64')
    tabla_final['nro_cuenta'] = tabla_final['nro_cuenta'].astype('int64')
    tabla_final['cod_producto'] = tabla_final['cod_producto'].astype('int64')

    tabla_final['fecha_apertura'] = pd.to_datetime(tabla_final['fecha_apertura'], format='%Y-%m-%d')
    tabla_final['fecha_ultima_transaccion'] = pd.to_datetime(tabla_final['fecha_ultima_transaccion'], format='%Y-%m-%d')
    
    # Filtro clientes y cast fechas
    tabla_final = tabla_final[tabla_final['identificacion'] != -1]
    tabla_final['fecha'] = pd.to_datetime(tabla_final['periodo'], format='%Y%m')
    
    return tabla_final


def get_pasivos_last_3(tabla_final):
    """Takes a product-level data frame and returns a client-level data frame,
    :param activos complete activos DataFrame
    :return: DataFrame, client-level
    """

    # Validación de duplicados a nivel 'id_cliente','nrooblig','fecha'
    pasivos_last_3 = tabla_final.sort_values(['identificacion','nro_cuenta','periodo'])
    # Se eliminan duplicados
    pasivos_last_3 = pasivos_last_3.drop_duplicates(subset=['identificacion','nro_cuenta','periodo'],keep='first')
    
    return pasivos_last_3


def create_pasivo_last_3_info(pasivos_last_3):
    """Creates aux variables requeried for the final explanatory variables
    :param activos to activos file location
    :return: DataFrame
    """
       
    # Se crea variable
    pasivos_last_3['fecha_pasivo'] = pasivos_last_3['periodo'].astype(int)
    pasivos_last_3['ano_po'] = pasivos_last_3['periodo'].astype('str').str[0:4].astype(int)

    # Se asigna el salario minimo mensual vigente de 2018 y 2019
    pasivos_last_3['SMLV'] = np.where(pasivos_last_3['ano_po'] == 2018, 781242,
                                            np.where(pasivos_last_3['ano_po'] == 2019, 828116,
                                                     np.where(pasivos_last_3['ano_po'] == 2020, 877803,
                                                              np.where(pasivos_last_3['ano_po'] == 2021, 908526,
                                                              np.nan))))
    
    # Variable para el delta de fechas
    pasivos_last_3['dia_fecha_corte'] = pasivos_last_3['fecha']+timedelta(days=37) ##TODO: Confirmar días

    return pasivos_last_3


def create_pasivos_df(pasivos_last_3):
    """Creates client-level final variables
    :param activos complete activos DataFrame
    :param activos_cli_fec containing activos relevant rows and columns and activos_cli_fec client level data base
    :return: DataFrame
    """

    pasivos_last_3_cli_fec = pd.DataFrame(pasivos_last_3.groupby(['tipo_id', 'identificacion', 'periodo'])[['SMLV','fecha_pasivo']].max())
    
    ## Agrupaciones
    gp_pasv = pasivos_last_3.groupby(['tipo_id','identificacion','periodo'])
    gp_pasv_ca = pasivos_last_3[pasivos_last_3['tipo_producto']=='ahorro'].groupby(['tipo_id','identificacion','periodo'])
    
    ## Antiguedad Total
    pasivos_last_3_cli_fec['pasv_antig_total'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                    gp_pasv['fecha_apertura'].min())/np.timedelta64(30, 'D')
    ## Numero meses ultima apertura
    pasivos_last_3_cli_fec['pasv_num_meses_ult_apertura'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                    gp_pasv['fecha_apertura'].max())/np.timedelta64(30, 'D')
    ## Dia última transacción
    pasivos_last_3_cli_fec['pasv_dias_desde_ultima_trans'] = (gp_pasv['dia_fecha_corte'].max() - 
                                                                    gp_pasv['fecha_ultima_transaccion'].max())/np.timedelta64(1, 'D')
    
    ## Saldo Total por cartera fin de mes
    pasivos_last_3_cli_fec['pasv_saldo_tot_fin_mes_smlv'] = (gp_pasv['saldo'].sum())/(gp_pasv['SMLV'].max())
    pasivos_last_3_cli_fec['pasv_saldo_ca_fin_mes_smlv'] = (gp_pasv_ca['saldo'].sum())/(gp_pasv_ca['SMLV'].max())

    # Creando fecha
    pasivos_last_3_cli_fec.reset_index(inplace=True)
    pasivos_last_3_cli_fec['fecha'] = pd.to_datetime(pasivos_last_3_cli_fec['periodo'], format='%Y%m')

    return pasivos_last_3_cli_fec


def create_pasv_out(pasivos_last_3_cli_fec):
    """Takes a product-level data frame and returns a client-level data frame,
    :param activos complete activos DataFrame
    :return: DataFrame, client-level
    """

    str_dates = ['202102','202103','202104'] #### Vector fechas

    for ix, date in enumerate(str_dates):
        # print(date)
        ids = pasivos_last_3_cli_fec.loc[pasivos_last_3_cli_fec['periodo']==int(date), ['tipo_id','identificacion']].drop_duplicates()

        if ix == 0:
            pasv_out = gp_func(ids, date, pasivos_last_3_cli_fec)
        else:    
            pasv_temp = gp_func(ids, date, pasivos_last_3_cli_fec)
            pasv_out = pd.concat([pasv_out, pasv_temp], ignore_index=True)
    
    return pasv_out


def pasivo_last_3_tot(pasivos_last_3_cli_fec, pasv_out):
    """Takes a product-level data frame and returns a client-level data frame,
    :param activos complete activos DataFrame
    :return: DataFrame, client-level
    """

    ## Set_index
    pasivos_last_3_cli_fec.set_index(['tipo_id','identificacion','periodo'], inplace=True)
    pasv_out.set_index(['tipo_id','identificacion','periodo'], inplace=True)
    
    # Join de las dos tablas creadas
    pasvivos_df = pasivos_last_3_cli_fec.join(pasv_out, how='left')
    
    # Selección de variables finales
    pasvivos_df.reset_index(inplace=True)
    pasvivos_df = pasvivos_df.drop(['SMLV','periodo','fecha','pasv_saldo_ca_fin_mes_smlv','pasv_saldo_tot_fin_mes_smlv'], axis=1)
    
    return pasvivos_df


def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe


def gp_func(ids, date, pasivos_last_3_cli_fec):
    fecha = pd.to_datetime(date, format='%Y%m')
    n_meses = [1]
    
    pasv_out = ids.set_index(['tipo_id','identificacion']) # this is different from the actual prod code
    
    for n in n_meses:
        col_name = 'pasv_saldo_total_'+str(n)+'mes_atras'
        col_name_ca = 'pasv_saldo_ca_'+str(n)+'mes_atras'
        date_past = fecha - timedelta(days=n*30)

        gp = pasivos_last_3_cli_fec[(pasivos_last_3_cli_fec['fecha']>date_past)&(pasivos_last_3_cli_fec['fecha']<=fecha)].groupby(['tipo_id','identificacion'])
        
        pasv_out[col_name] = gp['pasv_saldo_tot_fin_mes_smlv'].sum()/n
        pasv_out[col_name_ca] = gp['pasv_saldo_ca_fin_mes_smlv'].sum()/n
        
    pasv_out['periodo'] = int(date)
    #pasv_out.index.rename('id_cliente', inplace=True)
    pasv_out.reset_index(inplace=True)
    
    return pasv_out

In [18]:
# pasvivos_df = process_pasivos(input_paths_activos)
# print(pasvivos_df)
# pasvivos_df.head(6)

In [19]:
aaaa

NameError: name 'aaaa' is not defined

## Process Consolidado Premdt

In [ ]:
def consolidado_premdt(ids_final2):

    poblacion_total = ids_final2.drop_duplicates(subset=['id_numero_cliente','fecha_camp'], keep='first')
    poblacion_total['periodo'] = '07-2021'
    poblacion_total['periodo'].value_counts().sort_index()
    
    #Creación de fecha correspondiente para cruzar con la base campañas
    poblacion_total['data_camp'] = np.where((poblacion_total['periodo'] == '07-2021'), 202105, 199901)
    #Creación de fecha correspondiente para cruzar con la base buró
    poblacion_total['fecha_buro'] = np.where((poblacion_total['periodo'] == '07-2021'), 202101, 199901)
    #Creación de fecha correspondiente para cruzar con la base activo
    poblacion_total['fecha_activos'] = np.where((poblacion_total['periodo'] == '07-2021'), 202105, 199901)
    #Creación de fecha correspondiente para cruzar con la base pasivo
    poblacion_total['fecha_pasivo'] = np.where((poblacion_total['periodo'] == '07-2021'), 202105, 199901)
    #Creación de fecha correspondiente para cruzar con la base libranza
    poblacion_total['fecha_lib'] = np.where((poblacion_total['periodo'] == '07-2021'), 202106, 199901)
    #Creación de fecha correspondiente para cruzar con la base tx pasivo
    poblacion_total['fecha_tx_pav'] = np.where((poblacion_total['periodo'] == '07-2021'), 202105, 199901)
    
    # Cast ID
    poblacion_total = poblacion_total[poblacion_total['id_numero_cliente'].notnull()]
    poblacion_total['id_cliente'] = poblacion_total['id_cliente'].astype(int)
    poblacion_total['id_numero_cliente'] = poblacion_total['id_numero_cliente'].astype(int)
    poblacion_total = poblacion_total.drop_duplicates()
    
    return poblacion_total

In [ ]:
poblacion_total = consolidado_premdt(ids_final2)
print(poblacion_total.shape)
poblacion_total.head()

In [ ]:
input_paths = { 'libranzas': ['s3://data-bpop-dev-sandbox/estandarizado/productos/libranzas/productos_libranzas_dwh_M202104'],  ## Mensual 1 mes atras respecto al mes que esta corriendo - coger v2
         'clientes': ['s3://data-bpop-dev-sandbox/estandarizado/clientes/identificacion/clientes_identificacion_mdm_D20210218'],  ## Mensual 1 mes atras - coger v2 #### NO SE NECESITA QUITAR

         'activo': ['s3://data-bpop-dev-sandbox/estandarizado/productos/activo/productos_activo_sarc_M202104'],  ## Mensual 2 meses atras - coger v2

         'canpañas': 's3://data-bpop-dev-refined/gestion-campanas/preaprobado-tc/',
         'crm': ['s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20200228-20200601.csv',  ## Quemado
                 's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190801-20200201.csv',  ## Quemado
                 's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190104-20190702.csv'],  ## Quemado
         'vendors':['s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20200401-20200601.csv',  ## Quemado
                    's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20190101-20200301.csv'],  ## Quemado

         'activaciones': ['s3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-activadas/productos_activos-tarjeta-credito-activadas_H202003-202005.csv',  ## Quemado
                          's3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-activadas/productos_activos-tarjeta-credito-activadas_H20180101-20200327.csv'],  ## Quemado
         'acti_diarias': ['s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva'],  ##TodosDiarios

         'buro': 's3://data-bpop-dev-sandbox/estandarizado/riesgos-creditos/buro-comportamiento-financiero',  ##TodosDiarios

         'pasivo_cc': 's3://data-bpop-dev-sandbox/estandarizado/productos/pasivo-sfb-cc/productos_pasivo-sfb-cc_aaatr_M',

         'pasivo_cdt': 's3://data-bpop-dev-sandbox/estandarizado/productos/pasivo-sfb-cdts/productos_pasivo-sfb-cdts_aaatr_M',
                       
         'pasivo_ca': 's3://data-bpop-dev-sandbox/estandarizado/productos/pasivo-sfb-ca/productos_pasivo-sfb-ca_aaatr_M'
}

In [ ]:
path_file = get_s3_parquet_flag(input_paths['canpañas'], 0)[-2]

In [ ]:
path_file = input_paths['canpañas']

In [ ]:
path_file

In [ ]:
input_paths['canpañas'] + list_files(input_paths['canpañas'])[-1]

In [ ]:
def _key_files__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    return response.get("Contents", [])

In [ ]:
def get_s3_parquets(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    objects = _key_files__list(s3, bucket, key)
    files = ["s3://" + bucket + "/" + file["Key"][:-9] for file in objects if file["Key"].split("/")[-1]=="_SUCCESS"]
    return files

In [ ]:
session = boto3.session.Session()
s3 = session.client("s3")
bucket = s3_url.split("/")[2]
key = s3_url.split("/")[3:]
key = "/".join(key)
objects = _key_files__list(s3, bucket, key)
files = ["s3://" + bucket + "/" + file["Key"] for file in objects]# if file["Key"].split("/")[-1]=="_SUCCESS"]
#return files

In [ ]:
s3_url = input_paths['canpañas']

In [ ]:
s3_url

In [ ]:
session

In [ ]:
s3

In [ ]:
bucket

In [ ]:
key

In [ ]:
key

In [ ]:
objects

In [ ]:
files

In [ ]:
##5. ACTIVACIONES
def get_activaciones_file(input_paths):

    data_act2 = pd.read_csv(input_paths['activaciones'][0], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
    data_act2 = data_act2.drop_duplicates()
    print(data_act2.shape)
    
    data_act1 = pd.read_csv(input_paths['activaciones'][1], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
    data_act1 = data_act1.drop_duplicates()
    print(data_act1.shape)
    
    # Uniendo bases
    data_act2['fecha_activacion'] = data_act2['fecha_activacion'].str.replace('/20','/2020')
    data_act1 = pd.concat([data_act2,data_act1], ignore_index=True)
    
    # Casting variables
    data_act1['fecha_activacion'] = pd.to_datetime(data_act1['fecha_activacion'],format='%d/%m/%Y')
    data_act1['mes_activacion'] = pd.to_datetime(data_act1['fecha_activacion'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_act1.columns = data_act1.columns.str.lower()
    data_act1 = data_act1.rename(columns={'identificacion':'cedula'})
    del data_act2
    
    return data_act1

In [ ]:
data_act2 = pd.read_csv(input_paths['activaciones'][0], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
data_act2 = data_act2.drop_duplicates()
print(data_act2.shape)

In [ ]:
input_paths['activaciones'][0]

In [ ]:
list_files('s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/')

In [ ]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pandas_profiling
import os
from datetime import datetime, timedelta
import gc
from datetime import datetime
from dateutil.relativedelta import relativedelta

import boto3
import boto3.session

pd.set_option('display.max_columns',None)

import s3fs
import pyarrow.parquet as pq


def process_camp_activos(input_paths_activos):
    """Pending docstring"""
    
    # Pre_crm. from different route
    data_crm1 = get_crm_new_file(input_paths_activos)
    print('get_crm_new_file')
    # Pre_vend. from different route
    data_vend1 = get_vend_new_file(input_paths_activos)
    print('get_vend_new_file')
    
    # 1. Get crm file
    data_crm = get_crm_file(input_paths_activos)
    print('get_crm_file')
    # 2. Deletes duplicates and casts variables
    data_crm = get_crm_info(data_crm, data_crm1)
    print('get_crm_info')
    print(data_crm.head())
    
    # 3. Get vend file
    data_vend = get_vend_file(input_paths_activos)
    print('get_vend_file')
    # 4. Deletes duplicates, casts variables and union data_crm - data_vend
    data_camp = get_vend_info(data_vend, data_crm, data_vend1)
    print('get_vend_info')
    print(data_camp.head())
    
    # 5. Get activaciones file
    data_act1 = get_activaciones_file(input_paths_activos)
    print('get_activaciones_file')
    # 6. Get activaciones_diarias file
    data_act = get_acti_diarias_file(input_paths_activos)
    print('get_acti_diarias_file')
    # 7. Deletes duplicates and casts variables
    data_act = get_act_diarias_info(data_act, data_act1)
    print('get_act_diarias_info')
    
    # 8. Creates aux variables requeried for the final explanatory variables
    base_final = create_func_activa(data_camp, data_act)
    print('create_func_activa')
#    # 9. Deletes duplicates and casts variables
#    base_final = get_func_activa_info(base_final)
#    print('get_func_activa_info')
#    
#    # 10. Creates aux variables requeried for the final explanatory variables
#    tabla_final = create_func_var_camp_act(base_final)
#    print('create_func_var_camp_act')
#    # 11. Deletes duplicates and casts variables
#    tabla_final = get_func_var_camp_act(tabla_final)
#    print('get_func_var_camp_act')
#    print('camp_activos_df created successfully')  
    
    return base_final

#########################################################
################### Bases de campañas ###################
#########################################################

#####Prep. crm_new
def get_crm_new_file(input_paths):
    
    path_file = get_s3_parquet_flag(input_paths['canpañas'], 0)[-1]
    data_temp = pd.read_csv(path_file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str})
    data_temp = data_temp.sort_values(['base_name','date','clientid'])
    data_temp = data_temp.groupby(['clientid','base_name']).first()
    data_temp.reset_index(inplace=True)
    
    ## Filtrar express-la14
    data_temp = data_temp.loc[~data_temp['base_name'].str.contains('EXPRES_LA14')]
    data_temp = data_temp.rename(columns={'clientid':'NUM_DOC'})
    
    ## Oficinas
    data_crm = data_temp.loc[~data_temp['base_name'].str.contains('OFICINA_BOGOTA')]
    ## tipo campaña
    data_crm['TIPO CAMPAÑA'] = 'Preaprobados'
    data_crm['tipo_camp'] = 'of-preaprobados'
    # Casting variables
    data_crm['FECHA'] = pd.to_datetime(data_crm['date'],format='%Y-%m-%d')
    data_crm['mes_campaña'] = pd.to_datetime(np.where(data_crm['FECHA'].dt.day>=28, 
                                                      data_crm['FECHA']+timedelta(days=10),
                                                      data_crm['FECHA']),format='%Y-%m-%d')
    data_crm['mes_campaña2'] = pd.to_datetime(data_crm['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_crm['mes_campaña'] = pd.to_datetime(data_crm['mes_campaña'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_crm1 = data_crm.copy()
    
    return data_crm1


#####Prep. vend_new
def get_vend_new_file(input_paths):
    
    path_file = get_s3_parquet_flag(input_paths['canpañas'], 0)[-1]
    data_temp = pd.read_csv(path_file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str})
    data_temp = data_temp.sort_values(['base_name','date','clientid'])
    data_temp = data_temp.groupby(['clientid','base_name']).first()
    data_temp.reset_index(inplace=True)
    
    ## Filtrar express-la14
    data_temp = data_temp.loc[~data_temp['base_name'].str.contains('EXPRES_LA14')]
    data_temp = data_temp.rename(columns={'clientid':'NUM_DOC'})
    
    data_vend = data_temp.loc[~data_temp['base_name'].str.contains('CALL_ATENTO')]
    # Casting variables
    data_vend['TIPO CAMPAÑA'] = 'Preaprobados'
    data_vend['tipo_camp'] = 'at-preaprobados'
    data_vend['FECHA'] = pd.to_datetime(data_vend['date'],format='%Y-%m-%d')
    data_vend['mes_campaña'] = pd.to_datetime(data_vend['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_vend1 = data_vend.copy()
    
    return data_vend1


#####1. OFICINAS

def get_crm_file(input_paths):
    files = input_paths['crm']
    
    for ix, file in enumerate(files):
        print('File:', file)
        
        data_temp = pd.read_csv(file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str}) 
        
        if ix == 0:
            data_crm = data_temp
        else:
            data_crm = pd.concat([data_crm,data_temp], ignore_index=True)
            
        del data_temp
    
    return data_crm

## 2.    
## tipo campaña
def get_crm_info(data_crm, data_crm1):
    
    data_crm['TIPO CAMPAÑA'] = 'Preaprobados'
    data_crm['tipo_camp'] = 'of-preaprobados'
    
    # Casting variables
    data_crm['FECHA'] = pd.to_datetime(data_crm['FECHA'],format='%d/%m/%Y')
    data_crm['mes_campaña'] = pd.to_datetime(np.where(data_crm['FECHA'].dt.day>=28,
                                                      data_crm['FECHA']+timedelta(days=10), 
                                                      data_crm['FECHA']),format='%Y-%m-%d')
    data_crm['mes_campaña2'] = pd.to_datetime(data_crm['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_crm['mes_campaña'] = pd.to_datetime(data_crm['mes_campaña'].dt.strftime('%Y-%m'),format='%Y-%m')
    
    data_crm = pd.concat([data_crm1, data_crm], ignore_index=True)
    
    return data_crm


##### 3. ATENTO
def get_vend_file(input_paths):

    files = input_paths['vendors']
    
    for ix, file in enumerate(files):
        print('File:', file)
        
        data_temp = pd.read_csv(file, sep='|', encoding='ISO8859-1',quoting=1, dtype={'ID_CLIENTE':str,'NUM_DOC':str})
        if ix == 1:
            data_temp['FECHA2'] = pd.to_datetime(data_temp['FECHA'],format='%d/%m/%Y')
            data_temp = data_temp[data_temp['FECHA2']<'2020-12-01']
            data_temp = data_temp[['TIP_DOC','NUM_DOC','ID_CLIENTE',' CUPO_APROBADO ','FECHA','TIPO CAMPAÑA']]
        if ix == 0:
            data_vend = data_temp
        else:
            data_vend = pd.concat([data_vend,data_temp], ignore_index=True)
            
        del data_temp
    
    return data_vend


#4. Casting variables
def get_vend_info(data_vend, data_crm, data_vend1):
    
    data_vend['tipo_camp'] = np.where(data_vend['TIPO CAMPAÑA'] == 'Preaprobados','at-preaprobados','at-perfilados')
    data_vend['FECHA'] = pd.to_datetime(data_vend['FECHA'],format='%d/%m/%Y')
    data_vend['mes_campaña'] = pd.to_datetime(data_vend['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
    
    data_vend = pd.concat([data_vend1,data_vend], ignore_index=True)
    
    cols = ['NUM_DOC', 'FECHA', 'mes_campaña', 'TIPO CAMPAÑA', 'tipo_camp']
    
    data_vend.columns = data_vend.columns.str.strip()
    data_camp = pd.concat([data_crm[cols], data_vend[cols]], ignore_index=True)[cols]
        
    data_camp['NUM_DOC'] = data_camp['NUM_DOC'].astype(np.int64)
    data_camp['FECHA'] = pd.to_datetime(data_camp['FECHA'])
    data_camp['mes_campaña'] = pd.to_datetime(data_camp['mes_campaña'])
    data_camp = data_camp.rename(columns={'NUM_DOC':'cedula'})
    
    return data_camp

    
##5. ACTIVACIONES
def get_activaciones_file(input_paths):

    data_act2 = pd.read_csv(input_paths['activaciones'][0], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
    data_act2 = data_act2.drop_duplicates()
    print(data_act2.shape)
    
    data_act1 = pd.read_csv(input_paths['activaciones'][1], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
    data_act1 = data_act1.drop_duplicates()
    print(data_act1.shape)
    
    # Uniendo bases
    data_act2['fecha_activacion'] = data_act2['fecha_activacion'].str.replace('/20','/2020')
    data_act1 = pd.concat([data_act2,data_act1], ignore_index=True)
    
    # Casting variables
    data_act1['fecha_activacion'] = pd.to_datetime(data_act1['fecha_activacion'],format='%d/%m/%Y')
    data_act1['mes_activacion'] = pd.to_datetime(data_act1['fecha_activacion'].dt.strftime('%Y-%m'),format='%Y-%m')
    data_act1.columns = data_act1.columns.str.lower()
    data_act1 = data_act1.rename(columns={'identificacion':'cedula'})
    del data_act2
    
    return data_act1


##6. bases diarias
def get_acti_diarias_file(input_paths):

    files = get_s3_parquets(input_paths['acti_diarias'][0])
    
    for ix, file in enumerate(files):
        print('File:', file)
        data_temp = spark_read_parquet(file)
        if ix == 0:
            data_act = data_temp
        else:
            data_act = pd.concat([data_act,data_temp], ignore_index=True)
            
        del data_temp
        
    return data_act

        
#7. Casting variables     
def get_act_diarias_info(data_act, data_act1):

    data_act['fecha_activacion_plastico'] = pd.to_datetime(data_act['fecha_activacion_plastico'])
    data_act['mes_activacion'] = pd.to_datetime(data_act['fecha_activacion_plastico'].dt.strftime('%Y-%m'),format='%Y-%m')
    
    ## Renombrando variables
    data_act = data_act.rename(columns={'id_cli':'cedula',
                                       'tipo_id_cliente':'tipo_identificacion',
                                       'fecha_activacion_plastico':'fecha_activacion'})
    
    data_act = data_act[['numero_cuenta', 'numero_tarjeta', 'cedula', 'tipo_identificacion', 'fecha_activacion', 'id_cliente',
           'mes_activacion']]
    
    ## Uniendo bases
    data_act = pd.concat([data_act1, data_act], ignore_index=True)
    data_act = data_act.drop_duplicates()

    del data_act1

    ## Cast variables
    data_act['id_cliente'] = data_act['id_cliente'].fillna('-9999').astype(np.int64)
    data_act['tipo_identificacion'] = data_act['tipo_identificacion'].fillna('-9999').astype(np.int64)
    data_act['cedula'] = data_act['cedula'].astype(np.int64)

    return data_act


###8. func_activa
def create_func_activa(data_camp, data_act):

    str_dates = ['2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
                 '2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10', 
                 '2020-11','2020-12','2021-01','2021-02','2021-03','2021-04','2021-05','2021-06']
    n_mes = 3 # numero de meses
    
    for ix, date in enumerate(str_dates):
        print('Fecha:',date)
        date = pd.to_datetime(date,format='%Y-%m')
        nom_col = 'camp_'+date.strftime('%Y%m')
        
        base = data_camp[data_camp['mes_campaña']==date]
    
        if ix == 0:
            base_final = func_activa(base, data_act, date, n_mes, nom_col)
            del base
        else:
            base_temp = func_activa(base, data_act, date, n_mes, nom_col)
            base_final = pd.concat([base_final, base_temp], ignore_index=True)
            
            del base, base_temp

    return base_final

In [ ]:
base_final = process_camp_activos(input_paths)

In [ ]:
base_final.head()